In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import copy
import numpy as np
from numpy.random import *
from os import listdir as ld
from os.path import join as pj
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import visdom
from PIL import Image
import pandas as pd

# IO
from IO.utils import refine_result_by_ovthresh
from IO.loader import load_path, load_images
# utils
from utils.crop import crop_adjusted_std, crop_adjusted_std_resize
# Dataset
from dataset.detection.dataset import insects_dataset_from_voc_style_txt, collate_fn
# det model
from model.refinedet.refinedet import RefineDet
# Predict
from model.refinedet.utils.predict import test_prediction
# Evaluate
from evaluation.detection.evaluate import Voc_Evaluater, visualize_mean_index
from evaluation.detection.statistics import compute_each_size_df, compute_all_size_df
from evaluation.det2cls.visualize import vis_detections

# Test Config

In [ ]:
class args:
    # paths
    data_root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
    test_image_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_refined_images/20191106"
    test_target_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_detection_data/refinedet_all"
    det_model_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/output_model/detection/RefineDet", "crop_b2/tcb5_im512_freeze_20200806_use_extra_All_uniform120")
    figure_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/figure/detection/RefineDet", "crop_b2/tcb5_im512_freeze_20200806_use_extra_All_uniform120")
    save_img_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/figure/detection/RefineDet", "crop_b2/tcb5_im512_freeze_20200806_use_extra_All_uniform120")
    test_anno_folders = ["annotations_4"]
    # det model config
    input_size = 512 # choices=[320, 512, 1024]
    crop_num = (5, 5)
    tcb_layer_num = 6
    use_extra_layer = True
    det_activation_function = "ReLU"
    use_GN_WS = False
    # test_config
    eval_det2cls = False

In [ ]:
# class label
if args.eval_det2cls is True:
    args.det_labels = ['Coleoptera', 'Diptera', 'Ephemeridae', 
                       'Ephemeroptera', 'Hemiptera', 'Lepidoptera', 
                       'Plecoptera', 'Trichoptera', 'medium insect', 
                       'small insect', 'snail', 'spider']
else:
    args.det_labels = ['insects']

# Set cuda

In [ ]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# Detection Model

In [ ]:
det_model = RefineDet(args.input_size, len(args.det_labels)+1, args.tcb_layer_num, activation_function=args.det_activation_function, use_extra_layer=args.use_extra_layer, use_GN_WS=args.use_GN_WS)
load_name = pj(args.det_model_root, 'RefineDet{}_{}.pth'.format(args.input_size, "final"))
det_model.load_state_dict(torch.load(load_name))

### Make data

In [ ]:
print('Loading dataset for test ...')
test_dataset = insects_dataset_from_voc_style_txt(args.test_image_root, args.input_size, args.crop_num, training=False)
test_data_loader = data.DataLoader(test_dataset, 1, num_workers=0, shuffle=False, collate_fn=collate_fn)
print('Loading images ...')
anno_paths, image_paths = load_path(args.data_root, "refined_images", args.test_anno_folders)
images = load_images(image_paths)

# --- result analysis ---

In [ ]:
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
def get_det_result(det_model, data_loader, crop_num, num_classes, nms_thresh=0.3, ovthresh=0.3):
    det_result = test_prediction(det_model, data_loader, crop_num, num_classes, nms_thresh)
    det_result = refine_result_by_ovthresh(det_result, ovthresh)
    return det_result

In [ ]:
det_result = get_det_result(det_model, test_data_loader, args.crop_num, len(args.det_labels)+1)

In [ ]:
if args.eval_det2cls is True:
    # because ["annotations_4"] not include "Hemiptera"
    new_result = {}
    for image_id, result_per_image in det_result.items():
        result_per_image[4] = result_per_image[5]
        result_per_image[5] = result_per_image[6]
        result_per_image[6] = result_per_image[7]
        result_per_image[7] = result_per_image[8]
        result_per_image[8] = result_per_image[9]
        result_per_image[9] = result_per_image[10]
        result_per_image[10] = result_per_image[11]
        new_result.update({image_id: result_per_image})
    det_result = new_result

In [ ]:
evaluater = Voc_Evaluater(args.test_image_root, args.test_target_root, args.figure_root)
evaluater.set_result(det_result)
eval_metrics = evaluater.get_eval_metrics()

In [ ]:
if args.eval_det2cls is True:
    visualize_mean_index(eval_metrics, refinedet_only=True)
else:
    tp_fn = eval_metrics[0]['total positives']
    tp = eval_metrics[0]['total TP']
    fp = eval_metrics[0]['total FP']
    AP = eval_metrics[0]['AP']
    precision = tp/(tp + fp)
    recall = tp/tp_fn
    print("--- evaluation index for Other ---")
    print("AP = {}".format(AP))
    print("precision = {}".format(precision))
    print("recall = {}".format(recall))

### --- save AP, precision, recall ---

In [ ]:
class_df = pd.DataFrame({"AP": [AP], "precision": [precision], "recall": [recall]})
class_df.to_csv(pj(args.figure_root, "class_df.csv"))
class_df

### --- calculate size df ---

In [ ]:
each_df = compute_each_size_df(eval_metrics)
each_df.to_csv(pj(args.figure_root, "each_size_df.csv"))
each_df

In [ ]:
all_df = compute_all_size_df(each_df)
all_df.to_csv(pj(args.figure_root, "all_size_df.csv"))
all_df

# --- Output image with result ---

In [ ]:
if os.path.exists(args.save_img_root) is False:
    os.makedirs(args.save_img_root)

In [ ]:
def get_imagenames_from_anno_paths(anno_paths):
    imagenames = [anno_path.split('/')[-1] for anno_path in anno_paths]
    if '.ipynb_checkpoints' in imagenames:
        imagenames.remove('.ipynb_checkpoints')
    imagenames = [imagename.split('.')[0] for imagename in imagenames]
    return imagenames

In [ ]:
imagenames = get_imagenames_from_anno_paths(anno_paths)
colors = ["white", "red", "lime", "blue", "yellow", "fuchsia", "aqua", "gray", "maroon", "green", "navy", "olive", "purple", "teal"]
if args.eval_det2cls is True:
    insect_names = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                    'Lepidoptera', 'Plecoptera', 'Trichoptera']
    target_label = [1, 2, 3, 4, 5, 6]
else:
    insect_names = ['Insect']
    target_label = [0]

In [ ]:
def read_ground_truth(test_target_path, height, width):
    gt_coord = {}
    if args.eval_det2cls:
        lbl_array = range(11)
    else:
        lbl_array = range(1)
    
    for lbl in lbl_array:
        gt_coord.update({lbl: []})

    with open(test_target_path, mode="r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.split('\n')[0]
            elements = line.split(' ')
            target_lbl = int(elements[4])
            x1 = float(elements[0]) * width
            x2 = float(elements[2]) * width
            y1 = float(elements[1]) * height
            y2 = float(elements[3]) * height
            coord = [x1, y1, x2, y2]
            gt_coord[target_lbl].append(coord)
    
    for lbl in lbl_array:
        gt_coord[lbl] = np.asarray(gt_coord[lbl])
    
    return gt_coord

In [ ]:
def output_img_with_result(det_result, test_image_root, test_target_root, imagenames, insect_names, save_img_root):
    for i in range(len(imagenames)):
        print("output image: {}".format(imagenames[i]+".png"))
        img = np.asarray(Image.open(pj(test_image_root, imagenames[i]+".png")))
        height, width, _ = img.shape
        gt_coord = read_ground_truth(pj(test_target_root, imagenames[i]+".txt"), height, width)
        coord_per_image = copy.copy(det_result[imagenames[i]])
        for j, lbl in enumerate(target_label):
            img = vis_detections(img, coord_per_image[lbl], class_name=insect_names[j], color_name=colors[j])
            img = vis_detections(img, gt_coord[lbl], class_name=insect_names[j], color_name=colors[j])
        
        img = Image.fromarray(img)
        img.save(pj(save_img_root, imagenames[i]+".png"))

In [ ]:
output_img_with_result(det_result, args.test_image_root, args.test_target_root, imagenames, insect_names, args.save_img_root)